In [1]:
from google.colab import drive
drive.mount('drive')

ValueError: mount failed

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/MyDrive/CL/Day_3/Kopia heart_data_normalized.csv')
df

,sex_male,chestpain_nonanginal,chestpain_nontypical,chestpain_typical,fbs_yes,restecg_1,restecg_2,exang_yes,thal_normal,thal_reversable,ahd_yes,age,restbp,chol,maxhr,oldpeak,slope,ca
0,1,0,0,1,1,0,1,0,0,0,0,0.708333,0.481132,0.244292,0.603053,0.370968,1.0,0.000000
1,1,0,0,0,0,0,1,1,1,0,1,0.791667,0.622642,0.365297,0.282443,0.241935,0.5,1.000000
2,1,0,0,0,0,0,1,1,0,1,1,0.791667,0.245283,0.235160,0.442748,0.419355,0.5,0.666667
3,1,1,0,0,0,0,0,0,0,1,0,0.166667,0.339623,0.260274,0.885496,0.564516,1.0,0.000000
4,0,0,1,0,0,0,1,0,0,0,0,0.250000,0.339623,0.178082,0.770992,0.225806,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,1,0,0,1,0,0,0,0,0,1,1,0.333333,0.150943,0.315068,0.465649,0.193548,0.5,0.000000
299,1,0,0,0,1,0,0,0,0,1,1,0.812500,0.471698,0.152968,0.534351,0.548387,0.5,0.666667
300,1,0,0,0,0,0,0,1,0,1,1,0.583333,0.339623,0.011416,0.335878,0.193548,0.5,0.333333
301,0,0,1,0,0,0,1,0,1,0,1,0.583333,0.339623,0.251142,0.786260,0.000000,0.5,0.333333


In [4]:
df = pd.read_csv('/content/drive/MyDrive/CL/Day_3/Kopia diamonds_preprocessed.csv')
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,2,1,3,61.5,55.0,326,3.95,3.98,2.43
1,0.21,3,1,2,59.8,61.0,326,3.89,3.84,2.31
2,0.23,1,1,4,56.9,65.0,327,4.05,4.07,2.31
3,0.29,3,5,5,62.4,58.0,334,4.20,4.23,2.63
4,0.31,1,6,3,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53894,0.72,2,0,2,60.8,57.0,2757,5.75,5.76,3.50
53895,0.72,1,0,2,63.1,55.0,2757,5.69,5.75,3.61
53896,0.70,4,0,2,62.8,60.0,2757,5.66,5.68,3.56
53897,0.86,3,4,3,61.0,58.0,2757,6.15,6.12,3.74


In [5]:
from sklearn.model_selection import train_test_split
X = df.drop(['price'],axis=1)
y = df['price']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [6]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error

# Stworzenie instancji modelu XGBoost do regresji
xgb_regressor = XGBRegressor(random_state=42, n_jobs=-1)
# Definicja siatki parametrów
params = {
    'max_depth': [3, 4, 5, 6],
    'n_estimators': [100, 200, 300, 500],
    'min_child_weight': [1, 3, 5],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Stworzenie obiektu RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_regressor, param_distributions=params, scoring='neg_mean_absolute_error', cv=5, n_iter=10, verbose=1, random_state=42, n_jobs=-1)


In [7]:
# Trening modelu z RandomizedSearchCV
random_search.fit(X_train, y_train)

# Najlepszy zestaw hiperparametrów
best_params = random_search.best_params_
print(f"Najlepsze hiperparametry: {best_params}")

# Trening modelu z najlepszym zestawem hiperparametrów
best_model = random_search.best_estimator_


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Najlepsze hiperparametry: {'n_estimators': 300, 'min_child_weight': 5, 'max_depth': 5, 'learning_rate': 0.1}


In [8]:
# Predykcja na zbiorze testowym
y_pred = best_model.predict(X_test)

# Obliczenie średniego błędu bezwzględnego
mae = mean_absolute_error(y_test, y_pred)
print(f"Średni błąd bezwzględny: {mae}")


Średni błąd bezwzględny: 270.44429010923216
